#Import

In [1]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install seqeval

     |████████████████████████████████| 3.8 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 42.5 MB/s 
     |████████████████████████████████| 596 kB 45.1 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 6.5 MB 45.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 312 kB 5.5 MB/s 
     |████████████████████████████████| 212 kB 34.5 MB/s 
     |████████████████████████████████| 1.1 MB 29.5 MB/s 
     |████████████████████████████████| 134 kB 33.2 MB/s 
     |████████████████████████████████| 127 kB 39.1 MB/s 
     |████████████████████████████████| 271 kB 33.6 MB/s 
     |████████████████████████████████| 144 kB 11.2 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [ ]:
!pip3 install vncorenlp

In [2]:
from datasets import load_dataset, DatasetDict
from transformers import AdamW, AutoTokenizer, BertTokenizerFast, AutoModelForTokenClassification, AutoModel
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
import datasets
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import CrossEntropyLoss
# from vncorenlp import VnCoreNLP
import re 
from sklearn.metrics import f1_score
import string
from seqeval.metrics import classification_report
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Check GPU Memory

In [ ]:
!pip install gputil
!pip install psutil
!pip install humanize

  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=00fd8f84acf9e82bf4019990af24c7a3459828c0044ad83905cbc8be57df2b21
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built gputil


In [ ]:
# Import packages
import os,sys,humanize,psutil,GPUtil

# Define function
def mem_report():
  print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))
  
  GPUs = GPUtil.getGPUs()
  for i, gpu in enumerate(GPUs):
    print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))
    
# Execute function
mem_report()

CPU RAM Free: 11.9 GB
GPU 0 ... Mem Free: 11441MB / 11441MB | Utilization   0%


#Load pretrained model, dataset

In [ ]:
# annotator = VnCoreNLP("/content/drive/MyDrive/Code/NLP Research/file/VnCoreNLP-1.1.1.jar")

# # Input 
# text = "Ông Nguyễn Khắc Chúc  đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây."

# # To perform word segmentation, POS tagging, NER and then dependency parsing
# annotated_text = annotator.annotate(text)   

# # To perform word segmentation only
# word_segmented_text = annotator.tokenize(text)
# word_segmented_text

In [4]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
phobert = AutoModel.from_pretrained("vinai/phobert-base")

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
dataset = load_dataset('wikiann', 'vi')

Downloading:   0%|          | 0.00/3.94k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/234M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset wikiann downloaded and prepared to /root/.cache/huggingface/datasets/wikiann/vi/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
def group_token(example):
    for i in range(len(example)):
        token = example[i]
        if re.search('[a-zA-Z]', token) != None:
            token = token.translate(str.maketrans('', '', string.punctuation))
        example[i] = token
    res = ' '.join(example)
    return res 

def process_data(dataset):
    data_pd = pd.DataFrame(dataset)
    data_pd['sentence'] = data_pd['tokens'].apply(group_token)

    dataset_1 = datasets.arrow_dataset.Dataset.from_pandas(data_pd)
    return dataset_1
    

In [7]:
dataset_1 = process_data(dataset['train'])

TypeError: ignored

In [9]:
def check(examples):
    tokenized_examples = tokenizer(
                            examples['sentence'],
                            padding='max_length',
                            truncation=True,
                            max_length=30,
                            # return_offsets_mapping=True
                        )
    print(examples['sentence'])
    print(tokenizer.convert_ids_to_tokens(tokenized_examples['input_ids']))
    print(examples['ner_tags'])
def process_label(token_ids_list: list, label_list: list):
    processed_label = [-100] * len(token_ids_list)
    offset = []
    flag = 0
    start = 0
    end = 0
    for i in range(len(token_ids_list)):
        token_ids = token_ids_list[i]

        if token_ids[-2:] == '@@':
            # if re.search('[a-zA-Z0-9]', token_ids) != None:
                if flag == 1:
                    start = end
                    end += len(token_ids) - 2
                    
                else:
                    start = 0
                    end = len(token_ids) - 2
                    flag = 1
                offset.append([start, end])
            # else:
            #     offset.append([0, 0])
            #     flag = 1
        elif token_ids == '<s>' or token_ids == '</s>' or token_ids == '<pad>' or token_ids == '<unk>':
            offset.append([0, 0])
            continue
        else:
            if flag == 1:
                    start = end
                    end += len(token_ids)
                    flag = 0
            else:
                start = 0 
                end = len(token_ids)
                flag = 0
        
            offset.append([start, end])
    
    j = 0
    for idx, ofs in enumerate(offset):
        assert j <= len(label_list), f"j: {j} > len(label_list): {len(label_list)}"
        if ofs[0] == 0 and ofs[1] != 0:
            processed_label[idx] = label_list[j]
            j += 1
    # print("Offset's mapping's length: ", len(offset))
    # print("Offset: ", offset)
    # print("Number labels: ", len(label_list))
    
    # print("Offset           Token           Label")
    # for i in range(len(token_ids_list)):
    #     print(f'{offset[i]}           {token_ids_list[i]}           {processed_label[i]}')
    # print("j: ", j)
    return processed_label



In [ ]:
# for i in range(30):
#     print(f"------------Test {i+1} ------------")
#     example = dataset_1[i]
#     print(example['sentence'])
#     print(example['tokens'])
#     tokenized_examples = tokenizer(
#                                 example['sentence'],
#                                 padding='max_length',
#                                 truncation=True,
#                                 max_length=30,
#                                 # return_offsets_mapping=True
#                             )
#     # print(
#     #     process_label(tokenizer.convert_ids_to_tokens(tokenized_examples['input_ids']), 
#     #     example['ner_tags'])
#     # )
#     print(example['ner_tags'])
#     process_label(tokenizer.convert_ids_to_tokens(tokenized_examples['input_ids']), example['ner_tags'])
#     # print(check(example))

------------Test 1 ------------
Đồng bằng sông Cửu Long
['Đồng', 'bằng', 'sông', 'Cửu', 'Long']
[5, 6, 6, 6, 6]
Offset           Token           Label
[0, 0]           <s>           -100
[0, 4]           Đồng           5
[0, 4]           bằng           6
[0, 4]           sông           6
[0, 3]           Cửu           6
[0, 4]           Long           6
[0, 0]           </s>           -100
[0, 0]           <pad>           -100
[0, 0]           <pad>           -100
[0, 0]           <pad>           -100
[0, 0]           <pad>           -100
[0, 0]           <pad>           -100
[0, 0]           <pad>           -100
[0, 0]           <pad>           -100
[0, 0]           <pad>           -100
[0, 0]           <pad>           -100
[0, 0]           <pad>           -100
[0, 0]           <pad>           -100
[0, 0]           <pad>           -100
[0, 0]           <pad>           -100
[0, 0]           <pad>           -100
[0, 0]           <pad>           -100
[0, 0]           <pad>           -100

In [10]:
class NER_Dataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.len = len(data)
        self.data = data
        self.tokenizer = tokenizer 
        self.max_len = max_len
    def __getitem__(self, index, mode='train'):
        sentence = self.data['sentence'][index]
        word_labels = self.data['ner_tags'][index]

        encoding = self.tokenizer(
                            sentence,
                            padding='max_length', 
                            truncation=True, 
                            max_length=self.max_len
        )

        encoded_labels = process_label(
            token_ids_list= self.tokenizer.convert_ids_to_tokens(encoding['input_ids']),
            label_list= word_labels
        )

        

        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)

        return item
    def __len__(self):
        return len(self.data)

        

In [11]:
train_dataset = NER_Dataset(
    data= process_data(dataset['train']),
    tokenizer= tokenizer,
    max_len= 128
)
test_dataset = NER_Dataset(
    data= process_data(dataset['test']),
    tokenizer= tokenizer,
    max_len= 128
)

In [12]:
print(test_dataset.__len__())
print(train_dataset.__len__())

10000
20000


In [13]:
train_params = {'batch_size': 64,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': 64,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(train_dataset, **train_params)
testing_loader = DataLoader(test_dataset, **test_params)

In [14]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [15]:
class PhoBERT_NER(nn.Module):
    def __init__(self, bert, output_dim):
        super(PhoBERT_NER, self).__init__()
        self.bert = bert
        for params in self.bert.parameters():
            params.requires_grad = False
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        self.fc1 = nn.Linear(embedding_dim, 256)
        self.fc2 = nn.Linear(256, output_dim)
    def forward(self, input_ids, attention_mask):
        outputs, _ = self.bert(input_ids= input_ids, 
                            attention_mask= attention_mask,
                            return_dict= False
                            )
        # outputs = self.bert(input_ids=input_ids)
        # outputs = outputs[0]
        outputs = F.relu(self.fc1(outputs))
        outputs = self.fc2(outputs)
        return outputs

In [16]:
phobert_ner = PhoBERT_NER(phobert, 7)
phobert_ner = phobert_ner.to(device)

In [17]:
checkpoint = torch.load('/content/drive/MyDrive/Code/NLP Research/save/ner_2.pt')

In [18]:
phobert_ner.load_state_dict(checkpoint)
# checkpoint.keys()

<All keys matched successfully>

In [19]:
criterion = CrossEntropyLoss().to(device)
num_epochs = 5
lr = 1e-4
optimizer = optim.AdamW(phobert_ner.parameters(), lr=lr)


In [20]:
def compute_accuracy(pred, label, pad_idx):
    max_pred_idx = pred.argmax(dim=1, keepdim=True)
    tags_idx = (label != pad_idx).nonzero()
    correct = max_pred_idx[tags_idx].squeeze(1).eq(label[tags_idx])

    return correct.sum() / torch.FloatTensor([label[tags_idx].shape[0]]).to(device)
def compute_f1_score(pred, label, pad_idx):
    max_pred_idx = pred.argmax(dim=1, keepdim=True)
    tags_idx = (label != pad_idx).nonzero()

    return f1_score(label[tags_idx].cpu(), max_pred_idx[tags_idx].squeeze(1).cpu(), average='micro')

In [ ]:
x = ["Đại", "học", "Bách", "Khoa", "Hà", "Nội"]
y_true = ['O', 'O', 'O', 'O', 'O', 'O']

In [ ]:
# batch = next(iter(training_loader))
# print(batch['input_ids'].shape)


# compute_accuracy(output, label, pad_idx=-100)

torch.Size([64, 128])


In [ ]:
# torch.cuda.empty_cache()
# input_ids = batch['input_ids'].to(device)
# attention_mask = batch['attention_mask'].to(device)
# label = batch['labels'].to(device)
# print(label.shape)

# output = phobert_ner(input_ids, attention_mask)
# print(output.shape)
# output = output.contiguous().view(-1, output.shape[-1])
# print(output.shape)

# flattened_targets = label.view(-1)
# print(flattened_targets.shape)
# loss = criterion(output, flattened_targets)
# print(loss.item())

# loss.backward()
# optimizer.step()

torch.Size([64, 128])
torch.Size([64, 128, 7])
torch.Size([8192, 7])
torch.Size([8192])
1.9492393732070923


In [ ]:
# Sentence-level / Document-level
# Token-level , B-PERSON

In [ ]:
# output = phobert_ner(input_ids, attention_mask)
# print(output.shape)
# output = output.contiguous().view(-1, output.shape[-1])
# print(output.shape)

# flattened_targets = label.view(-1)
# print(flattened_targets.shape)
# loss = criterion(output, flattened_targets)
# print(loss.item())

torch.Size([64, 128, 7])
torch.Size([8192, 7])
torch.Size([8192])
1.9143092632293701


In [21]:
from tqdm import tqdm

In [26]:
def train(model, training_loader, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    epoch_f1 = 0
    model.train()

    for idx, batch in tqdm(enumerate(training_loader)):
        optimizer.zero_grad()

        ids = batch['input_ids'].to(device, dtype=torch.long)
        att_mask = batch['attention_mask'].to(device, dtype=torch.long)
        labels = batch['labels'].to(device, dtype=torch.long)

        flattend_targets = labels.view(-1)
        

        outputs = model(ids, att_mask)
        outputs = outputs.contiguous().view(-1, outputs.shape[-1])
        labels = labels.contiguous().view(-1)

        loss = criterion(outputs, labels)
        acc = compute_accuracy(outputs, labels, -100)
        # f1 = compute_f1_score(outputs, labels, -100) 

        loss.backward()
        optimizer.step()

        print ('loss train: {}, acc train: {}'.format(loss.item(), acc.item()))
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        # epoch_f1 += f1
    return epoch_loss / len(training_loader), epoch_acc / len(training_loader)
    
def eval(model, testing_loader, criterion):
    epoch_loss = 0
    epoch_acc = 0
    epoch_f1 = 0
    model.eval()

    y_true = []
    y_pred = []

    with torch.no_grad():
        for idx, batch in tqdm(enumerate(testing_loader)):

            ids = batch['input_ids'].to(device, dtype=torch.long)
            att_mask = batch['attention_mask'].to(device, dtype=torch.long)
            labels = batch['labels'].to(device, dtype=torch.long)

            flattend_targets = labels.view(-1)
            

            outputs = model(ids, att_mask)
            outputs = outputs.contiguous().view(-1, outputs.shape[-1])
            labels = labels.contiguous().view(-1)


            active_labels = labels != -100

            y_true.extend(torch.masked_select(labels, active_labels).cpu().numpy())
            y_pred.extend(torch.masked_select(torch.argmax(outputs, dim=1), active_labels).cpu().numpy())

            loss = criterion(outputs, labels)
            acc = compute_accuracy(outputs, labels, -100)
             
            print ('loss test: {}, acc test: {}'.format(loss.item(), acc.item()))
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            # epoch_f1 += f1


        
    return epoch_loss / len(testing_loader), epoch_acc / len(testing_loader), y_true, y_pred

In [ ]:
full_train_loss, full_train_acc, full_train_f1_score = [], [], []
full_test_loss, full_test_acc, full_test_f1_score = [], [], []
for epoch in range(num_epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch+1, num_epochs))
    print('---TRAINING PHASE---')
    train_loss, train_acc = train(
        model=phobert_ner,
        training_loader= training_loader,
        optimizer= optimizer,
        criterion= criterion
    )
    full_train_loss.append(train_loss)
    full_train_acc.append(train_acc)
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    
    path = '/content/drive/MyDrive/Code/NLP Research/save/ner_2.pt'
    torch.save(phobert_ner.state_dict(), path)
    print("Model saved!")
    # print('---TESTING PHASE---')
    # test_loss, test_acc, test_f1_score = eval(
    #     model=phobert_ner,
    #     testing_loader= testing_loader,
    #     criterion= criterion
    # )
    # full_test_loss.append(test_loss)
    # full_test_acc.append(test_acc)
    # print(f'\tTest Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Test F1 Score: {test_f1_score}')

======== Epoch 1 / 5 ========
---TRAINING PHASE---


1it [00:13, 13.05s/it]

loss train: 1.914312481880188, acc train: 0.23115578293800354


2it [00:22, 11.04s/it]

loss train: 1.8984073400497437, acc train: 0.2685851454734802


...

loss train: 0.524203896522522, acc train: 0.820045530796051


133it [21:18,  9.94s/it]

loss train: 0.5049212574958801, acc train: 0.8263888955116272


134it [21:28,  9.94s/it]

loss train: 0.5115232467651367, acc train: 0.8374999761581421


In [27]:
print('---TESTING PHASE---')
test_loss, test_acc, y_true, y_pred = eval(
    model=phobert_ner,
    testing_loader= testing_loader,
    criterion= criterion
)
print(f'\tTest Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

---TESTING PHASE---


1it [00:04,  4.66s/it]

loss test: 0.3387638032436371, acc test: 0.8865740895271301


2it [00:09,  4.67s/it]

loss test: 0.4704957604408264, acc test: 0.8426666855812073


...

loss test: 0.6147196292877197, acc test: 0.8028503656387329


156it [12:02,  4.64s/it]

loss test: 0.5262791514396667, acc test: 0.8208616971969604


157it [12:03,  4.61s/it]

loss test: 0.4159806966781616, acc test: 0.8681318759918213
	Test Loss: 0.475 | Test Acc: 84.34%


In [47]:
label_to_idx = {"O": 0, "B-PER": 1, "I-PER": 2, "B-ORG": 3, "I-ORG": 4, "B-LOC": 5, "I-LOC": 6}
idx_to_label = {0: "O", 1: "B-PER", 2: "I-PER", 3: "B-ORG", 4: "I-ORG", 5: "B-LOC", 6: "I-LOC"}

# sample usage
y_true_1 = [idx_to_label[i] for i in y_true]
y_pred_1 = [idx_to_label[i] for i in y_pred]

print("Confusion matrix: ", confusion_matrix(y_true, y_pred))
print("F1 Score of each class: ", f1_score(y_true, y_pred, average=None))

Confusion matrix:  [[22461   133   167   233   679   229   268]
 [   86  3334    86   217    24   129     8]
 [  389    70  6161    11   819    26   310]
 [  261   215    26  2762   169   260    10]
 [ 1035    63   564   179 10864    56   800]
 [  113   130    32   285   103  2843   210]
 [  262     6   224     7  1172    72  6274]]
F1 Score of each class:  [0.92096685 0.85105297 0.8189552  0.74678924 0.79325326 0.77561042
 0.78933132]


In [ ]:
example = test_dataset.__getitem__(1)

predict(example, phobert_ner).shape

tensor([[   0,    0,    0,    0,    0,    0,    0,    1,    2,    2,    0,    0,
            0,    0,    0,    0, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100]], device='cuda:0')
torch.Size([1, 128, 7])
torch.Size([128, 7])
tensor([[ 0,  0],
        [ 0,  1],
        [ 0,  2],
        [ 0,  3

RuntimeError: ignored

In [ ]:
# Relation Extraction
# Event Detection, Event Argument Extraction
# Generation (Machine Translation, Summarization)

In [ ]:
# 1. Sửa lại code và phần đánh giá
# 2. Đọc về GCN: https://www.cs.mcgill.ca/~wlh/grl_book/
# 3. Relation Extraction